In [ ]:
! pip install kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
#https://www.kaggle.com/code/bettycxh06/rri-amplitudes/input
!kaggle datasets download -d bettycxh06/apnea-ecg-database

!kaggle kernels pull bettycxh06/rri-amplitudes


100% 314M/315M [00:12<00:00, 29.2MB/s]
100% 315M/315M [00:12<00:00, 26.5MB/s]
Source code downloaded to /content/rri-amplitudes.ipynb


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
os.listdir(os.getcwd())

['.config',
 'apnea-ecg-database.zip',
 'kaggle.json',
 'rri-amplitudes.ipynb',
 'sample_data']

In [ ]:
!unzip apnea-ecg-database.zip

Archive:  apnea-ecg-database.zip
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/ANNOTATORS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/RECORDS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/SHA256SUMS.txt  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.dat  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01r.apn  

In [ ]:
!pip install biosppy
!pip install wfdb

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import posixpath
import csv
import wfdb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 3.6 MB/s eta 0:00:00


In [ ]:
import pickle
import sys
from concurrent.futures import ProcessPoolExecutor, as_completed

import biosppy.signals.tools as st
import numpy as np
import os
import wfdb
from biosppy.signals.ecg import correct_rpeaks, hamilton_segmenter
from scipy.signal import medfilt
from tqdm import tqdm

# PhysioNet Apnea-ECG dataset
# url: https://physionet.org/physiobank/database/apnea-ecg/
#base_dir = "../input/apnea-ecg-database/apnea-ecg-database-1.0.0"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"



fs = 100
sample = fs * 60  # 1 min's sample points

before = 2  # forward interval (min)
after = 2  # backward interval (min)
hr_min = 20
hr_max = 300

num_worker = 35


def worker(name, labels):
    X = []
    y = []
    groups = []
    signals = wfdb.rdrecord(os.path.join(base_dir, name), channels=[0]).p_signal[:, 0]
    for j in tqdm(range(len(labels)), desc=name, file=sys.stdout):
        if j < before or \
                (j + 1 + after) > len(signals) / float(sample):
            continue
        signal = signals[int((j - before) * sample):int((j + 1 + after) * sample)]
        signal, _, _ = st.filter_signal(signal, ftype='FIR', band='bandpass', order=int(0.3 * fs),
                                        frequency=[3, 45], sampling_rate=fs)
        # Find R peaks
        rpeaks, = hamilton_segmenter(signal, sampling_rate=fs)
        rpeaks, = correct_rpeaks(signal, rpeaks=rpeaks, sampling_rate=fs, tol=0.1)
        if len(rpeaks) / (1 + after + before) < 40 or \
                len(rpeaks) / (1 + after + before) > 200:  # Remove abnormal R peaks signal
            continue
        # Extract RRI, Ampl signal
        rri_tm, rri_signal = rpeaks[1:] / float(fs), np.diff(rpeaks) / float(fs)
        rri_signal = medfilt(rri_signal, kernel_size=3)
        ampl_tm, ampl_siganl = rpeaks / float(fs), signal[rpeaks]
        hr = 60 / rri_signal
        # Remove physiologically impossible HR signal
        if np.all(np.logical_and(hr >= hr_min, hr <= hr_max)):
            # Save extracted signal
            X.append([(rri_tm, rri_signal), (ampl_tm, ampl_siganl)])
            y.append(0. if labels[j] == 'N' else 1.)
            groups.append(name)
    return X, y, groups


if __name__ == "__main__":
    apnea_ecg = {}

    names = [
        "a01", "a02", "a03", "a04", "a05", "a06", "a07", "a08", "a09", "a10",
        "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20",
        "b01", "b02", "b03", "b04", "b05",
        "c01", "c02", "c03", "c04", "c05", "c06", "c07", "c08", "c09", "c10"
    ]

    o_train = []
    y_train = []
    groups_train = []
    print('Training...')
    with ProcessPoolExecutor(max_workers=num_worker) as executor:
        task_list = []
        for i in range(len(names)):
            labels = wfdb.rdann(os.path.join(base_dir, names[i]), extension="apn").symbol
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_train.extend(X)
            y_train.extend(y)
            groups_train.extend(groups)

    print()



    answers = {}
    with open(os.path.join("event-2-answers"), "r") as f:
    #    with open(os.path.join(base_dir, "event-2-answers"), "r") as f:
        for answer in f.read().split("\n\n"):
            answers[answer[:3]] = list("".join(answer.split()[2::2]))

    names = [
        "x01", "x02", "x03", "x04", "x05", "x06", "x07", "x08", "x09", "x10",
        "x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20",
        "x21", "x22", "x23", "x24", "x25", "x26", "x27", "x28", "x29", "x30",
        "x31", "x32", "x33", "x34", "x35"
    ]

    o_test = []
    y_test = []
    groups_test = []
    print("Testing...")
    with ProcessPoolExecutor(max_workers=num_worker) as executor:

        task_list = []
        for i in range(len(names)):
            labels = answers[names[i]]
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_test.extend(X)
            y_test.extend(y)
            groups_test.extend(groups)

    apnea_ecg = dict(o_train=o_train, y_train=y_train, groups_train=groups_train, o_test=o_test, y_test=y_test,
                     groups_test=groups_test)
    with open(os.path.join(base_dir, "apnea-ecg.pkl"), "wb") as f:
        pickle.dump(apnea_ecg, f, protocol=2)

    print("\nok!")

Training...
a17: 100%|██████████| 485/485 [54:54<00:00,  6.79s/it]

Testing...
x12: 100%|██████████| 527/527 [54:27<00:00,  6.20s/it]

ok!


In [ ]:
!pip install keras
!pip install tensorflow
#
#IMPORT LIBRARIES
import pickle
import numpy as np
import tensorflow
import os
from keras.callbacks import LearningRateScheduler,EarlyStopping
from keras.layers import Dense,Flatten,MaxPooling2D,Conv2D,BatchNormalization
from keras.regularizers import l2
from scipy.interpolate import splev, splrep
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
import keras
import matplotlib.pyplot as plt
from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.models import Model, load_model, save_model
from keras.layers import Input
import pandas as pd

#base_dir = "dataset"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"

#------------------------------------------------------------------------------
# DEEP LEARNING MODELS NEED TO VECTORS OR MATRIX WITH SAME SIZE
# R-R INTERVALS DONT HAVE SAME SIZE, SO WE NEED TO INTERPOLATE VECTORS TO GET VECTORS WITH SAME SIZE.
# BASED ON OUR EXPERIENCE INTERPOLATION IN 3 HZ BETTER AND ACCURATE.
ir = 3 # INTERPOLATION RATE(3HZ)
time_range= 60 # 60-s INTERVALS OF ECG SIGNALS
weight=1e-3 #  WEIGHT L2 FOR REGULARIZATION(AVODING OVERFITTING PARAMETER)
#-----------------------------
# NORMALIZATION:
# DEEP LEARNING AND EVEN NEURAL NETWORKS INPUT SHOULD BE NORMALIZED:
# MIN-MAX METHOD APPLIED FOR SCALING:(Array-min(Array))/(max(Array)-min(Array))
scaler = lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr))
#-----------------------------
# FIRSTLY WE PRE-PROCESSED OUR DATA IN "apnea-ecg.pkl" FILE
# IN PRE-PROCESSING SECTION WE EXTRACT R-R INTERVALS AND R-PEAK AMPLITUDES
# IN THIS PART WE LOAD THIS DATA AND INTERPOLATE AND CONCATE FOR FEEDING TO NETWORKS
def load_data():
    tm = np.arange(0, (before + 1 + after) * 60, step=1 / float(ir))

    with open(os.path.join(base_dir, "apnea-ecg.pkl"), 'rb') as f: # read preprocessing result
        apnea_ecg = pickle.load(f)
#-----------------
    x_train = []
    o_train, y_train = apnea_ecg["o_train"], apnea_ecg["y_train"]

    for i in range(len(o_train)):
        (rri_tm, rri_signal), (amp_tm, amp_signal) = o_train[i]
      # Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        amp_interp_signal = splev(tm, splrep(amp_tm, scaler(amp_signal), k=3), ext=1)
        x_train.append([rri_interp_signal, amp_interp_signal])
    x_train = np.array(x_train, dtype="float32")

    x_train = np.expand_dims(x_train,1)
    x_train=np.array(x_train, dtype="float32").transpose((0,3,1,2)) # convert to numpy format

    #return x_train_final, y_train
    return x_train, y_train

In [ ]:
!pip install Hyperactive
!pip install mealpy
!pip install tensorflow
!pip install optimizers
!pip install keras

#from keras.layers.rnn import GRU
from keras.layers import Reshape, Lambda
from keras.layers import GRU
from hyperactive import Hyperactive
from hyperactive.optimizers import HillClimbingOptimizer
from hyperactive.optimizers import RandomSearchOptimizer
from mealpy.swarm_based.GWO import OriginalGWO

from tensorflow.keras import optimizers
from keras.optimizers import Adam
import threading
import concurrent.futures
from threading import Thread
import time
from tabnanny import verbose
from mealpy.swarm_based.HBA import OriginalHBA

from tensorflow.python.keras import optimizers
#from keras.optimizers import RandomSearchOptimizer

import warnings
warnings.filterwarnings('ignore')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.0/378.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for optimizers: filename=Optimizers-0.1-py3-none-any.whl size=17005 sha256=1db466abe93438f25379cff502dde4226188b8bd3d326c3caa44f587ccb26e8f
  Stored in directory: /root/.cache/pip/wheels/21/85/73/6114e547e45ab65253df3cba8144782286634fa9773a5a7828
Successfully built optimizers


In [ ]:
#print(x_train.shape, y_train.shape)
#x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
#x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
#x_train= x_train.transpose((0,2,1))
#print(x_train.shape, y_train.shape)
##(16709, 900, 1, 2) (16709, 2)
##(16709, 2, 900) (16709, 2)


In [ ]:
# BHA
###########################
def fitness_function(position):
    learning_rate,n1,n2,n3,n4,n5,n6,epochs = position[0],position[1],position[2],position[3],position[4],position[5],position[6],position[7]
    n1,n2,n3,n4,n5,n6,epochs=int(n1),int(n2),int(n3),int(n4),int(n5),int(n6),int(epochs)

    print(learning_rate,n1,n2,n3,n4,n5,n6,epochs)

#def create_model(weight=1e-3):
    model= Sequential()
    model.add(Conv2D(n1, kernel_size=(7,1), strides=(1,1), padding="valid", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight),input_shape=(900,1,2)))
    model.add(MaxPooling2D(pool_size=(3,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(n2, kernel_size=(5,1), strides=1, padding="valid", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(MaxPooling2D(pool_size=(3,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(n3, kernel_size=(3,1), strides=(1,1), padding="valid", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(Conv2D(n4, kernel_size=(3,1), strides=1, padding="valid", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(Conv2D(n5, kernel_size=(3,1), strides=1, padding="valid", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(MaxPooling2D(pool_size=(3,1),strides=(2,1)))
    model.add(Flatten())
    model.add(Dense(n6, activation="relu"))
    model.add(Dense(2, activation="softmax"))

    return learning_rate,n1,n2,n3,n4,n5,n6,epochs


    optimizer =keras.optimizers.Adam(lr=learning_rate)
    print("Learning rate: ", learning_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    #model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=['accuracy'])

    history=model.fit(X1, Y1, batch_size=128, epochs=int(epochs), validation_data=(x_val, y_val),
                       callbacks=[callback1,lr_scheduler])


    #history = model.fit(x_train,y_train, epochs=int(epochs),batch_size=32, validation_data=(x_test, y_test))
    return history.history['val_accuracy'][-1]
    #loss, accuracy = model.evaluate(x_test, y_test)

    #return model
#------------------------------------------------------------------------------
# Define learning rate schedule for preventing overfitting in deep learning methods:
def lr_schedule(epochs, learning_rate):
   if epochs > 70 and \
           (epochs - 1) % 10 == 0:
        learning_rate *= 0.1
   print("Learning rate: ", learning_rate)
   return learning_rate

#------------------------------------------------------------------------------

        # we used k-fold cross-validation for more reliable experiments:
   kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)
   cvscores = []
   ACC=[]
   SN=[]
   SP=[]
   F2=[]
   print("train num:", len(y_train))
   print("test num:", len(y_test))


    # separate train& test and then compile model
  # for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

     # Compile and evaluate model:
if __name__ == "__main__":
    # loading Data:
    x_train, y_train = load_data()
    # we have labels(Y) in a binary way 0 for normal and 1 for apnea patients
    # we want to classify data into 2-class so we changed y in a categorical way:
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
    # we used k-fold cross-validation for more reliable experiments:
    kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)


    # separate train& test and then compile model
    ##for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

    # define callback for early stopping:
    lr_scheduler = LearningRateScheduler(lr_schedule)
    callback1 = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
    x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
    x_train= x_train.transpose((0,2,1))
    #10% of Data used for validation:
    X1,x_val,Y1,y_val=train_test_split(x_train,y_train,test_size=0.10)

##################################################################
#pso = Pso(swarmsize=4,maxiter=14)
# n,sf,sp,l
##bp,value = pso.run(func,[1,2,2,2],[16,8,4,4])
#v = func(bp);
##################################################################
problem_dict1 = {
    "fit_func": fitness_function,
    'lb':[0.001 ,76,128,412,924,412,318,30], # Lower bound of our parameters
    'ub':[0.1   ,116,356,612,1128,612,518,200], # upper bound of our parameters
    "minmax": "min",
    "obj_weights": [0.4, 1, 1,1,1,1,1,1]               # Define it or default value will be [1, 1, 1]   [0.4, 0.1, 0.5,1,1,1,1,1]
    #"mode":"thread"
    #"amend_position": amend_position

                }

rng = np.random.default_rng()
epoch = 100
pop_size =50

#def thread_function(k):

  #run=k+1
  #print("Run Number",run)
  #Optimizer = MHoneyBadgerAlgorithm(search_config, n_iter=100, n_part=10, metric='accuracy', cv=10, h_beta=6.0, h_c=2.0,run=run)
model = OriginalGWO(epoch, pop_size)
  #t1 = time.time()

#best_position, best_fitness = model.solve(problem_dict1)
best_position, best_fitness= model.solve(problem_dict1)
#best_position, best_fitness= model.solve(mode="thread")
v = fitness_function(best_position);
#print(f"Solution: {best_position}, Fitness: {best_fitness}")
#return v
print(f"Solution: {best_position}, Fitness: {v}")

0.05438084670883477 95 242 428 1115 559 391 139


INFO:mealpy.swarm_based.GWO.OriginalGWO:Solving 8-objective optimization problem with weights: [0.4 1.  1.  1.  1.  1.  1.  1. ].


0.09672589731491327 83 314 559 941 427 361 122
0.006321412155941431 112 242 462 925 413 491 75
0.06157496515431319 108 274 520 1089 478 487 64
0.09501135289423211 103 248 507 1091 590 416 46
0.02490026583608549 107 203 601 999 533 357 54
0.07372335018603852 96 265 490 1122 465 451 104
0.08860531644307122 81 310 519 1038 575 334 96
0.006536700320024036 90 148 501 1112 561 363 141
0.058436251708717094 98 174 535 1061 569 420 48
0.04997549067127707 102 156 608 1037 577 376 115
0.07163098034129978 97 317 468 949 498 414 199
0.009199487826055328 107 294 420 969 475 328 165
0.07968388552015272 83 330 577 1040 507 389 171
0.03531738048013979 88 301 517 974 594 412 196
0.02212391321115074 108 172 434 1086 556 321 186
0.046862432837176234 86 173 505 1000 480 412 57
0.07094828548297945 101 147 469 1084 530 457 108
0.09750323676099096 98 351 579 969 468 471 163
0.007100970483968938 96 255 554 1098 487 369 98
0.06125428567807223 105 147 611 1052 579 436 46
0.04996181464791047 98 273 500 931 543 40

INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 1, Current best: 2424.0004, Global best: 2424.0004, Runtime: 12.41224 seconds


0.054119374400012715 116 177 412 924 443 318 50
0.1 110 128 412 1004 412 376 30
0.08676902449389517 76 270 427 924 440 318 77
0.05888232067790964 89 128 485 1128 412 320 179
0.01633641430886319 76 164 412 1128 612 318 30
0.05131702833901697 88 158 612 924 612 408 66
0.09521875401186793 76 128 412 1128 412 318 140
0.06593239078995511 88 221 585 1128 555 377 49
0.08851378004673426 116 128 412 1128 425 318 78
0.0790783754903615 76 135 490 1128 412 318 54
0.06453067794935884 116 145 444 924 412 318 60
0.033361518982213116 97 199 412 1037 506 318 92
0.07302235834726772 76 128 612 971 612 318 83
0.001 76 356 412 1128 572 318 30
0.006960307549416261 116 181 412 1128 412 489 86
0.010694536303814763 116 178 567 924 446 318 84
0.001 76 128 453 924 412 343 34
0.05131731661485866 76 148 412 1128 612 318 30
0.08008442902697732 76 128 412 924 582 480 57
0.06508043138366976 76 128 612 955 412 318 86
0.1 94 132 412 1128 412 318 104
0.001 81 128 412 1107 503 318 30
0.09622673978545886 88 204 412 1128 4

...

0.001 76 128 412 924 412 318 30
0.0010013618140379034 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 413 924 413 318 30
0.001 76 128 412 924 412 318 30
0.0010222380371993107 76 128 412 924 412 318 30
0.001070110427061946 76 128 416 924 412 318 30
0.0010034379256619764 76 128 412 924 413 318 30
0.0010025909489812105 76 128 412 924 412 318 30
0.001 76 128 412 924 413 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 319 30
0.001006316401185085 76 128 412 925 412 318 30
0.001 76 128 412 924 415 318 30
0.001010058722933256 76 128 412 928 413 318 30
0.001 76 128 412 924 412 318 30
0.0010012404772195586 76 128 412 927 412 318 30
0.0010023813365553783 76 128 412 924 412 319 30
0.0010053352403512396 76 128 413 924 412 318 30
0.0010015849598017542 76 128 412 926 412 318 30
0.0010000856689889412 76 128 412 924 412 318 30
0.0010001676891264858 76 128 412 926 412 31

INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 99, Current best: 2300.0004, Global best: 2300.0004, Runtime: 6.88992 seconds


0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76 128 412 924 412 318 30
0.001 76

INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 100, Current best: 2300.0004, Global best: 2300.0004, Runtime: 5.85074 seconds


0.001 76 128 412 924 412 318 30
Solution: [1.00e-03 7.60e+01 1.28e+02 4.12e+02 9.24e+02 4.12e+02 3.18e+02 3.00e+01], Fitness: (0.001, 76, 128, 412, 924, 412, 318, 30)
